In [2]:
!nvidia-smi #this command acts as a task manager between nvidia gpu and the user. Shows GPU related details (system management information)




Fri Oct 10 03:02:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
%cd /content/drive/MyDrive
#switching the directory to my drive to keep all the model related data there

/content/drive/MyDrive


In [6]:
!git clone https://github.com/Tanishk-Singh/Transformers.git
# cloning the git repo in my directory

fatal: destination path 'Transformers' already exists and is not an empty directory.


In [12]:
%cd Transformers
# switching to transofrmer git repo as directory

/content/drive/MyDrive/Transformers


In [13]:
!git config user.name "Tanishk-Singh"
!git config user.email "tanwartanishk5@gmail.com"

In [ ]:
!git status

Refresh index: 100% (5/5), done.
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   AWQ_Implementation.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [14]:
!pip install llmcompressor transformers accelerate

# original autoawq is depricated and now llcompressor by VLLM maintains it
# transformers - hugging face library to download latest llm models
# accelerate - hugging face library to load large models efficently on cpu/gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.2/273.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.0 MB/s eta 0:00:00


In [15]:
import torch

In [16]:
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print("AutoAWQ imported successfully!")

PyTorch: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4
AutoAWQ imported successfully!


In [18]:
!mkdir -p models/awq_quantized
#creates a folder name awq_quantised in directory models, p is the parent flag, if parent directory doesn't exist
# it creates one

In [30]:
from llmcompressor import oneshot
from transformers import AutoModelForCausalLM, AutoTokenizer



In [21]:
model_path = "facebook/opt-1.3b" # looks for this model in cache, if not found, loads it from Hugging Face Library to cache
output_dir = "/content/drive/MyDrive/Transformers/models/opt-1.3b-awq"
#to store our quantised or compresses model

print("Loading FP16 model...")
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16
)

Loading FP16 model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
# we first load tokeniser becasue they convert text to numbers to input into the model

fp16_size = sum(p.numel() * p.element_size() for p in model.parameters()) / 1e9
# here we iterate every tensor(weight matrix) in the model. Get the no of elements in each and mutiply it by size
# then convert total bytest to Gigabytes
print(f"Size of FP16 Parameters loaded: {fp16_size:.2f} GB")


Size of FP16 Parameters loaded: 2.63 GB


In [33]:
output_dir = "/content/drive/MyDrive/Transformers/models/opt-1.3b-awq"

print("\nQuantizing with AWQ (W4A16)...")

# AWQ quantization recipe
recipe = """
quant_stage:
    quant_modifiers:
        QuantizationModifier:
            ignore: ["lm_head"]
            config_groups:
                group_0:
                    targets: ["Linear"]
                    weights:
                        num_bits: 4
                        type: "int"
                        symmetric: True
                        strategy: "group"
                        group_size: 128
"""

oneshot = oneshot(
    model=model,
    dataset="open_platypus",
    num_calibration_samples=512,
    recipe=recipe,
    output_dir=output_dir
)

print(f"✅ Quantization complete! Saved to {output_dir}")


Quantizing with AWQ (W4A16)...


Tokenizing:   0%|          | 0/24926 [00:00<?, ? examples/s]

2025-10-10T04:22:59.502662+0000 | reset | INFO - Compression lifecycle reset
2025-10-10T04:22:59.559559+0000 | initialize | INFO - Compression lifecycle initialized for 1 modifiers
2025-10-10T04:22:59.560691+0000 | IndependentPipeline | INFO - Inferred `DataFreePipeline` for `QuantizationModifier`


Updating global scales: 100%|██████████| 144/144 [00:00<00:00, 316551.25it/s]
Fusing global scales: 416it [00:00, 288592.53it/s]
Calibrating weights: 100%|██████████| 144/144 [00:01<00:00, 89.00it/s] 

2025-10-10T04:23:03.875921+0000 | finalize | INFO - Compression lifecycle finalized for 1 modifiers


2025-10-10T04:23:05.432470+0000 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Compressing model: 144it [00:05, 27.85it/s]


TypeError: Module.apply() missing 1 required positional argument: 'fn'

In [38]:
# Load quantized model
quant_model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/Transformers/models/opt-1.3b-awq",
    device_map="auto"
)

# Calculate size of quanitsed model
quant_size = sum(p.numel() * p.element_size() for p in quant_model.parameters()) / 1e9

print(f"Model Size Comparison:")
print(f"FP16:  {fp16_size:.2f} GB")
print(f"AWQ:   {quant_size:.2f} GB")
print(f"Ratio: {fp16_size/quant_size:.1f}x smaller")

Compressing model: 144it [00:00, 1119.58it/s]


📊 Model Size Comparison:
FP16:  2.63 GB
AWQ:   1.04 GB
Ratio: 2.5x smaller


In [39]:
!git status

Refresh index: 100% (5/5), done.
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   AWQ_Implementation.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	models/
	sparse_logs/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add AWQ_Implementation.ipynb
!git commit -m "Compression for "
!git push

[main df52f47] Fixed notebook metadata for GitHub rendering
 1 file changed, 1556 insertions(+), 1 deletion(-)
 rewrite AWQ_Implementation.ipynb (99%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 5.37 KiB | 550.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/Tanishk-Singh/Transformers.git
   8c6960a..df52f47  main -> main


In [ ]:
!git push

Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 24.13 KiB | 2.19 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/Tanishk-Singh/Transformers.git
   42f942b..3a859df  main -> main
